In [45]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import json
train_idx = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camId = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelist = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idx = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idx = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
with open('PR_data/feature_data.json', 'r') as f:
    features = np.array(json.load(f))
train_idx -= 1
gallery_idx -= 1
query_idx -= 1
max_val = np.amax(features)
features = np.divide(features, max_val)

In [46]:
train_features = features[train_idx.tolist()]
gallery_features = features[gallery_idx.tolist()]
query_features = features[query_idx.tolist()]

train_label = labels[train_idx.tolist()]
gallery_label = labels[gallery_idx.tolist()]
query_label = labels[query_idx.tolist()]

train_cam = camId[train_idx.tolist()]
gallery_cam = camId[gallery_idx.tolist()]
query_cam = camId[query_idx.tolist()]

## Bagging functions

In [47]:
#zip together the training features, thier labels and cameras
labeled_train = np.asarray(list(zip(train_features, train_label, train_cam)))
print(labeled_train.shape)

(7368, 3)


In [48]:
import random as rand
from more_itertools import locate

in_triples = np.empty((7368, 3), int)
out_pairs =  np.empty((7368, 2), int)
for i in range(labeled_train.shape[0]) :

    
    current_id = labeled_train[i][1]
    current_cam = labeled_train[i][2]
    #ensure there are three unique ID's randomly selected
    rand_id1 = rand.choice(train_label)
    rand_id2 = rand.choice(train_label)
    while not(rand_id1 != current_id) and not(rand_id2 != current_id) and not(rand_id1 != rand_id2) :
        rand_id1 = rand.choice(train_label)
        rand_id2 = rand.choice(train_label)

#     print(current_id, rand_id1, rand_id2)

    triple_set = list(locate(labeled_train, lambda x: 
                        (x[1] == current_id and x[2] != current_cam)  
                        or (x[1] == rand_id1 and x[2] != current_cam)
                        or (x[1] == rand_id2 and x[2] != current_cam)
                            ))
#     print(triple_set)

    triple_1 = rand.choice(triple_set)
    triple_2 = rand.choice(triple_set)
    while triple_1 == triple_2 : 
        triple_1 = rand.choice(triple_set)
        triple_2 = rand.choice(triple_set)

    in_triples[i] = [i, triple_1, triple_2]

    out_pairs[i] = [(current_id == labeled_train[triple_1][1]),(current_id == labeled_train[triple_2][1])]
    


In [49]:
print(in_triples[0:10])
print(out_pairs[0:10])

[[   0    7  473]
 [   1 6198 6199]
 [   2  130  330]
 [   3    6    8]
 [   4    5 3805]
 [   5 1953    3]
 [   6 3601 2179]
 [   7 5590  606]
 [   8    4    1]
 [   9    0 4479]]
[[1 0]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [1 0]]


In [50]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=120,metric='manhattan')
classifier.fit(gallery_features,gallery_label)
values = classifier.kneighbors(query_features)

In [6]:
rankN = range(1,101)
ranks = [0] * len(rankN)

for i in range(len(query_idx)):
    cam_match = 0
    for j in range(len(values[1][i])):
        if gallery_label[values[1][i][j]] == query_label[i] and gallery_cam[values[1][i][j]] != query_cam[i]:
            
            for k in range(len(rankN)):
                if j - cam_match <= rankN[k]-1:
                    ranks[k] += 1
            break
            
        elif gallery_label[values[1][i][j]] == query_label[i] and gallery_cam[values[1][i][j]] == query_cam[i]:
            cam_match += 1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
for i in range(len(ranks)) :
    print("Rank: ", rankN[i], "  Accuracy: ", ranks[i]/len(query_label))

Rank:  1   Accuracy:  0.47214285714285714
Rank:  2   Accuracy:  0.5478571428571428
Rank:  3   Accuracy:  0.5985714285714285
Rank:  4   Accuracy:  0.6371428571428571
Rank:  5   Accuracy:  0.6614285714285715
Rank:  6   Accuracy:  0.6878571428571428
Rank:  7   Accuracy:  0.7028571428571428
Rank:  8   Accuracy:  0.7207142857142858
Rank:  9   Accuracy:  0.7378571428571429
Rank:  10   Accuracy:  0.7507142857142857
Rank:  11   Accuracy:  0.7592857142857142
Rank:  12   Accuracy:  0.7735714285714286
Rank:  13   Accuracy:  0.7842857142857143
Rank:  14   Accuracy:  0.79
Rank:  15   Accuracy:  0.7978571428571428
Rank:  16   Accuracy:  0.8
Rank:  17   Accuracy:  0.8057142857142857
Rank:  18   Accuracy:  0.8121428571428572
Rank:  19   Accuracy:  0.8228571428571428
Rank:  20   Accuracy:  0.8264285714285714
Rank:  21   Accuracy:  0.8292857142857143
Rank:  22   Accuracy:  0.8342857142857143
Rank:  23   Accuracy:  0.8385714285714285
Rank:  24   Accuracy:  0.8421428571428572
Rank:  25   Accuracy:  0.845
